#### 1. Definição do Problema

### Predição de Desligamentos de Funcionários

O desligamento de funcionários é definido como o processo natural pelo qual os funcionários deixam a força de trabalho – por exemplo, por meio de demissão por motivos pessoais ou aposentadoria – e não são substituídos imediatamente.

[Mais informações sobre "Employee attrition" podem ser encontradas aqui](https://www.betterup.com/blog/employee-attrition)

Este é um conjunto de dados fictício criado por cientistas de dados da IBM.

O objetivo é prever os desligamentos a partir dos dados fornecidos.

# 2. Extração dos Dados

#### Importação de Bibliotecas

In [92]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, accuracy_score, f1_score
from category_encoders import TargetEncoder
import warnings
warnings.filterwarnings("ignore")
plt.style.use('ggplot')

#### Extração dos Dados

In [93]:
# Extração dos Dados de treino
df = pd.read_csv('../input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')
# Exibição das 5 Primeiras Linhas do DataFrame
df.head()

# 3. Preparação dos Dados

In [94]:
# Nomes das colunas
df.columns

In [95]:
# Dimensão do Dataset
print('Número de Linhas = ',df.shape[0])
print('Número de Colunas = ',df.shape[1])

In [96]:
# Tipo de Cada Coluna
df.dtypes

In [97]:
unique_values = []
# Verificando os valores únicos por coluna
[(unique_values.append(str(df[coluna].nunique())),print(coluna+' possui '+str(df[coluna].nunique())+' valores únicos')) for coluna in df.columns];

In [98]:
# Verificando o único valor de EmployeeCount 
print(df.EmployeeCount.unique())
# Dropando esta coluna
df.drop('EmployeeCount',axis=1,inplace=True)

In [99]:
# Verificando o único valor de Over18
print(df.Over18.unique())
# Dropando esta coluna
df.drop('Over18',axis=1,inplace=True)

In [100]:
# Verificando o único valor de StandardHours
print(df.StandardHours.unique())
# Dropando esta coluna
df.drop('StandardHours',axis=1,inplace=True)

In [101]:
features_bar = []
# Seleção de variáveis para plotagem do gráfico de barras na análise univariada
[(features_bar.append(coluna)) for coluna in df.columns if df[coluna].nunique() <= 7];

In [102]:
# Verificando a existência de possíveis linhas duplicadas nos dados de treino
df.duplicated().sum()

In [103]:
# Dropando algumas colunas
df.drop(columns=['MonthlyRate','DailyRate'],axis=1,inplace=True)

In [104]:
# Verificando a existência de possíveis valores ausentes
df.isnull().sum().sort_values(ascending=False)

# 4. Exploração / Visualização dos Dados

#### Estatística Descritiva

In [105]:
# Estatística Descritiva

# O comando describe retorna parâmetros estatísticos tais como: contagem de linhas, média, desvio 
# padrão, mínimo, máximo, primeiro, segundo e terceiro quartis.
# Deve-se lembrar que este comando só se aplica às variáveis numéricas.

df_describe = df.describe().T

Embora bastante útil, o comando describe não expressa alguns parâmetros estatísticos relevantes para a análise de dados, tais como: skewness, kurtosis, mediana e intervalo.

df.skew() retorna um valor que nos indica como está a distribuição dos dados para aquela váriavel ou seja, se os dados estão simétricos ou assimétricos, caso estejam assimétricos podemos saber se está para esquerda ou para a direita. Se a assimetria (skew) estiver entre -0,5 e 0,5, os dados são bastante simétricos; Se a assimetria (skew) estiver entre -1 e -0,5 ou entre 0,5 e 1, os dados serão moderadamente distorcidos; Se a assimetria (skew) for menor que -1 ou maior que 1, os dados são altamente distorcidos;

Kurtosis é uma medida que caracteriza o achatamento da curva da função de distribuição de probabilidade, de tal forma que para uma distribuição normal, o valor de Kurtosis é 3. Frequentemente utiliza-se como parâmetro o excesso de kurtosis, obtido apenas realizando a subtração do valor de kurtosis por 3.

Se o excesso de kurtosis for igual a 0 significa dizer que se trata de uma distribuição normal;

Se o excesso de kurtosis for maior do que 0 significa dizer que a distribuição é mais concentrada do que a distribuição normal;

Se o excesso de kurtosis for menor do que 0 significa dizer que a distribuição é menos concentrada do que a distribuição normal;

O comando df.kurtosis() retorna o excesso de kurtosis;

A mediana separa a amostra em duas partes de modo que ela se encontra no centro da amostra. Para obtê-la deve-se ordenar a amostra em ordem crescente ou decrescente e buscar pelo valor que se localiza no centro da amostra.

O comando df.median() retorna a mediana de cada variável pertencente ao DataFrame df.

O intervalo representa a distância entre os dois extremos em uma amostra, ou seja, valor máximo subtraído pelo valor mínimo.

In [106]:
# Definindo as variáveis numéricas e categóricas
num_attributes = df.select_dtypes(include=['int64', 'float64'])
cat_attributes = df.select_dtypes(exclude=['int64', 'float64'])

In [107]:
df_skewness  = pd.DataFrame(num_attributes.skew(),columns=['skew'])
df_kurtosis  = pd.DataFrame(num_attributes.kurtosis(),columns=['kurtosis'])
df_median    = pd.DataFrame(num_attributes.median(),columns=['median'])
df_intervalo = pd.DataFrame(num_attributes.max() - num_attributes.min(),columns=['range'])

In [108]:
df_train_descritive_statistics = pd.concat([df_describe,df_skewness,df_kurtosis,df_median,df_intervalo],axis=1)
df_train_descritive_statistics

In [109]:
df_rescisao = df[df.Attrition == 'Yes']
df_ativo = df[df.Attrition != 'Yes']

In [110]:
# Afim de visualizar a distribuição dos dados para cada variável e identificar possíveis outliers utilizaremos a seguinte função que auxilia na plotagem dos gráficos boxplot
def plot_boxplot(df):
    # Retirar a variável Id de num_attributes
    numerical_columns = num_attributes.columns
    fig, ax = plt.subplots(int(len(numerical_columns)/2),2, 
                           figsize=(20,50))
    colunas = numerical_columns
    indice = 0
    for j in range(len(ax)):
        for i in range(len(ax[j])):
            ax[j][i].tick_params(labelsize=10)
            ax[j][i].set_title(colunas[indice])
            ax[j][i] = sns.boxplot(data=df[colunas[indice]], ax=ax[j][i])
            indice += 1
    fig.suptitle('Boxplots', position=(.5,1), fontsize=20)
    fig.tight_layout()
    return
# KDE plot 
plot_boxplot(df)

#### Qual a Quantidade de Desligamentos por Departamento ?

In [111]:
perc_department = (df_rescisao.groupby('Department').Attrition.count()).sort_values(ascending=False)[:10].reset_index().rename(columns={'Attrition':'quantidade'})
# Plotando
plt.figure(figsize=(10,5))
sns.barplot(x=perc_department.quantidade,y=perc_department.Department).set_title('Quantidade de desligamentos por Departamento')
plt.xlabel('Quantidade')
plt.ylabel('Departamento');
plt.show()

**O departamento de Pesquisa e Desenvolvimento teve a maior quantidade de desligamentos.**

#### Qual o Percentual de Desligados por Departamento ?

In [112]:
perc_department = (100*(df_rescisao.groupby('Department').Attrition.count()/df_ativo.groupby('Department').Attrition.count()).sort_values(ascending=False)[:10]).reset_index().rename(columns={'Attrition':'percentual'})
# Plotando
plt.figure(figsize=(10,5))
sns.barplot(x=perc_department.percentual,y=perc_department.Department).set_title('Percentuais de desligados por Departamento')
plt.xlabel('Percentual')
plt.ylabel('Departamento');
plt.show()

**Notamos que o departamento de vendas apresenta o maior percentual de profissionais desligados.**

#### Qual o Percentual de Desligados por Gênero?

In [113]:
perc_gender = (100*(df_rescisao.groupby('Gender').Attrition.count()/df_ativo.groupby('Gender').Attrition.count()).sort_values(ascending=False)[:10]).reset_index().rename(columns={'Attrition':'percentual'})
# Plotando
plt.figure(figsize=(10,5))
sns.barplot(x=perc_gender.percentual,y=perc_gender.Gender).set_title('Percentuais de desligados por Gênero')
plt.xlabel('Percentual')
plt.ylabel('Gênero')
plt.show()

**O gênero masculino apresenta o maior percentual de profissionais desligados.**

#### Qual a relação entre a Renda Mensal e Nível de Educação?

##### Funcionários que foram desligados tinham menor renda mensal?

In [114]:
plt.figure(figsize=(15,7))
sns.barplot(data=df,x='Education',y='MonthlyIncome',hue='Attrition').set_title('Renda Mensal por Nível de Educação');
plt.ylabel('Renda Mensal')
plt.xlabel('Nível de Educação')
plt.show()

Para melhor compreender o signicado de cada um dos valores da variável "Education" note a tabela a seguir:

`Education`:

`1` - 'Below College' |
`2` - 'College' |
`3` - 'Bachelor' |
`4` - 'Master' |
`5` - 'Doctor' |

**Notamos que à medida que o nível de educação é aumentado, há um aumento na renda mensal. Além disso, percebemos também que funcionários que foram desligados tinham renda mensal inferior aos funcionários que não foram desligados da empresa.**

#### Existe alguma relação entre o campo de educação e a renda mensal?

In [115]:
plt.figure(figsize=(15,7))
sns.kdeplot(x='MonthlyIncome',data=df,shade=True,hue='EducationField',alpha=0.10);

**Notamos que a média de renda mensal para o campo educacional de marketing é levemente superior a renda mensal dos demais campos educacionais.**

#### Qual a distribuição de Funcionários por Campos Educacionais e Departamentos?

In [116]:
pd.crosstab(df.Department,df.EducationField)

**Vemos que todos os profissionais de marketing fazem parte do departamento de vendas.**

In [117]:
# Aplicando coeficiente de correlação de Pearson
df_corr = df.corr()

In [118]:
# Visualizar a correlação entre cada variável utilizando a correlação de Pearson
_ = plt.figure(figsize = (30,20))
res = sns.heatmap(df_corr, vmax = 1, linewidths = 0.7, cmap = 'viridis',annot_kws={"size": 15},annot=True)
_ = plt.title('Pearson Correlation', fontsize = 20, pad = 15)
_ = res.set_yticklabels(res.get_ymajorticklabels(), fontsize = 14)
_ = res.set_xticklabels(res.get_xmajorticklabels(), fontsize = 14)

O coeficiente de correlação de Pearson nos auxiliar a identificar a intensidade e a direção da relação linear entre duas variáveis. Este coeficiente pode assumir valores entre -1 e 1, o valor em módulo indica a intensidade da relação linear entre as duas variáveis, enquanto o sinal indica a direção. Dessa forma, o sinal positivo indica correlação direta e o sinal negativo indica correlação inversa.

Em virtude do módulo e do sinal de cada um destes valores do coeficiente de correlação, inferimos que as principais correlações são entre:

`JobLevel e MonthlyIncome`: Indicando que um aumento na qualificação do funcionário(JobLevel) é acompanhada por um aumento na renda mensal(MonthlyIncome); 

`PerformanceRating e PercentSalaryHike`: Indicando que um aumento no rating de performance(PerformanceRating) é acompanhado por um acréscimo no percentual de aumento de salário(PercentSalaryHike); 

`JobLevel e TotalWorkingYears`: Indicando que um aumento na qualificação do funcionário(JobLevel) é acompanhado por um acréscimo no total de anos trabalhando(TotalWorkingYears); 

`YearsatCompany e YearsWithCurrManager`: Indicando que um aumento na quantidade de anos na empresa(YearsatCompany) é acompanhada por um aumento na quantidade de anos sendo liderada pelo atual gerente(YearsWithCurrManager); 

`TotalWorkingYears e MonthlyIncome`: Indicando que um aumento no total de anos trabalhando(TotalWorkingYears) é acompanhada por um aumento na renda mensal(MonthlyIncome); 

`YearsatCompany e YearsinCurrentRole`: Indicando que um aumento na quantidade de anos na empresa(YearsatCompany) é acompanhada por um aumento na quantidade de anos na mesma função(YearsinCurrentRole); 

**Deve-se lembrar que o coeficiente de correlação não implica causalidade.**

In [187]:
plt.figure(figsize=(15,5))
sns.scatterplot(data=df,x='JobLevel',y='MonthlyIncome').set_title('Nível de Qualificação do Funcionário e Renda Mensal');

Percebe-se que assim como visto na matriz de correlação, o aumento no nível de qualificação do funcionário é acompanhado pelo crescimento da renda mensal.

In [189]:
plt.figure(figsize=(15,5))
sns.scatterplot(data=df,x='PerformanceRating',y='PercentSalaryHike').set_title('Nível de performance e Aumento percentual no Salário');

Percebe-se que assim como visto na matriz de correlação, o aumento no nível de performance é acompanhado pelo crescimento do percentual do salário.

In [190]:
plt.figure(figsize=(15,5))
sns.scatterplot(data=df,x='JobLevel',y='TotalWorkingYears').set_title('Nível de Qualificação do Funcionário e Total de Anos Trabalhados');

Percebe-se que assim como visto na matriz de correlação, o aumento no nível de qualificação do funcionário é acompanhado pelo aumento no total de anos trabalhados.

In [194]:
plt.figure(figsize=(15,5))
sns.scatterplot(data=df,x='YearsAtCompany',y='YearsWithCurrManager').set_title('Anos na Companhia e Anos liderados pelo Gerente Atual');

Percebe-se que assim como visto na matriz de correlação, o aumento nos anos de companhia é acompanhado pelo aumento dos anos liderados pelo gerente atual.

In [196]:
plt.figure(figsize=(15,5))
sns.scatterplot(data=df,x='TotalWorkingYears',y='MonthlyIncome').set_title('Total de Anos Trabalhados e Renda Mensal');

Percebe-se que assim como visto na matriz de correlação, o aumento do total de anos trabalhados é acompanhado pelo crescimento da renda mensal.

In [200]:
plt.figure(figsize=(15,5))
sns.scatterplot(data=df,x='YearsAtCompany',y='YearsInCurrentRole').set_title('Anos na Companhia e Anos na Função Atual');

Percebe-se que assim como visto na matriz de correlação, o aumento nos anos de companhia é acompanhado pelo aumento na quantidade de anos na função atual.

# Modelo de Machine Learning

### Target Encoding

In [119]:
df.replace({'Yes': 1, 'No': 0},inplace=True)

In [120]:
# Verificando contagem de valores para as categorias da variável target
sns.barplot(x=df.Attrition.value_counts().index,y=df.Attrition.value_counts().values).set_title('Contagem Target');

Nota-se claramente um grande desbalanceamento nos dados, o que pode comprometer significantemente o desempenho do modelo de machine learning. Dessa forma, se torna necessário realizar o tratamento deste problema com o objetivo de desenvolver um modelo com melhor desempenho.

### Sem tratamento para os Dados Desbalanceados

In [121]:
# Realizando split dos dados
df_original = df.copy()
df_target_encoding = df.copy()
X = df_target_encoding.drop('Attrition',axis=1)
y = df_target_encoding.Attrition
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.30,random_state=0)

In [122]:
# Concatenando os dados de treino para aplicar o encoding em seguida
df_target_encoding_train = pd.concat([Xtrain,ytrain],axis=1)
# Concatenando os dados de treino para aplicar o encoding em seguida
df_target_encoding_test = pd.concat([Xtest,ytest],axis=1)

In [123]:
# Definindo as variáveis numéricas e categóricas
num_attributes = Xtrain.select_dtypes(include=['int64', 'float64'])
cat_attributes = Xtrain.select_dtypes(exclude=['int64', 'float64'])

In [124]:
encoder = TargetEncoder(cols=cat_attributes.columns)
Xtrain = encoder.fit_transform(df_target_encoding_train.drop('Attrition',axis=1),df_target_encoding_train.Attrition);
Xtest  = encoder.transform(df_target_encoding_test.drop('Attrition',axis=1),df_target_encoding_test.Attrition);
ytrain = df_target_encoding_train.Attrition
ytest = df_target_encoding_test.Attrition

In [125]:
param_grid = {'C': np.arange(0.01,0.05,0.01)}
search_logistic = GridSearchCV(LogisticRegression(penalty='l1',solver='liblinear'), param_grid)
search_logistic.fit(Xtrain,ytrain)
predlogistic = search_logistic.predict(Xtest)

In [126]:
# Média de acurácia
search_logistic.score(Xtest, ytest) 

In [127]:
# Matriz de confusão
matriz_confusao = confusion_matrix(ytest, predlogistic)
pd.DataFrame(matriz_confusao,index=['Real negativo','Real Verdadeiro'],columns=['Previsto Negativo','Previsto Positivo'])

In [128]:
# Outras métricas
print(classification_report(ytest, predlogistic))

In [129]:
# Computa probabilidades
y_pred_prob = search_logistic.predict_proba(Xtest)[:,1]

# Gera fpr, tpr e thresholds
fpr, tpr, thresholds = roc_curve(ytest, y_pred_prob)
auc_value = roc_auc_score(ytest, y_pred_prob)
plt.plot(fpr, tpr, color='blue', label='ROC curve (area = %0.2f)' % auc_value)
# curva ROC
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

## Random Undersampling

#### Target Encoding

In [130]:
df_target_encoding = df_original.copy()
X = df_target_encoding.drop('Attrition',axis=1)
y = df_target_encoding.Attrition
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.30,random_state=0)

# Concatenando os dados de treino para aplicar o encoding em seguida
df_target_encoding_train = pd.concat([Xtrain,ytrain],axis=1)
# Concatenando os dados de treino para aplicar o encoding em seguida
df_target_encoding_test = pd.concat([Xtest,ytest],axis=1)

# Definindo as variáveis numéricas e categóricas
num_attributes = Xtrain.select_dtypes(include=['int64', 'float64'])
cat_attributes = Xtrain.select_dtypes(exclude=['int64', 'float64'])

encoder = TargetEncoder(cols=cat_attributes.columns)
Xtrain = encoder.fit_transform(df_target_encoding_train.drop('Attrition',axis=1),df_target_encoding_train.Attrition);
Xtest  = encoder.transform(df_target_encoding_test.drop('Attrition',axis=1),df_target_encoding_test.Attrition);
ytrain = df_target_encoding_train.Attrition
ytest = df_target_encoding_test.Attrition
# Concatenando os dados de treino para aplicar o encoding em seguida
df_target_encoding_train = pd.concat([Xtrain,ytrain],axis=1)
# Concatenando os dados de treino para aplicar o encoding em seguida
df_target_encoding_test = pd.concat([Xtest,ytest],axis=1)

In [131]:
data = df_target_encoding_train.copy()
# class count
class_count_0, class_count_1 = data['Attrition'].value_counts()

# Separate class
class_0 = data[data['Attrition'] == 0]
class_1 = data[data['Attrition'] == 1]# print the shape of the class
print('Attrition 0:', class_0.shape)
print('Attrition 1:', class_1.shape)

In [132]:
class_0_under = class_0.sample(class_count_1)

test_under = pd.concat([class_0_under, class_1], axis=0)

# Verificando contagem de valores para as categorias da variável target após a realização do undersampling
sns.barplot(x=test_under['Attrition'].value_counts().index,y=test_under['Attrition'].value_counts().values).set_title('Contagem Target - Após Undersampling');

In [133]:
df = test_under.copy()
# Realizando split dos dados
df_target_train = df.copy()

In [134]:
Xtrain_uds = df_target_train.drop('Attrition',axis=1)
ytrain_uds = df_target_train.Attrition

In [135]:
# Definindo as variáveis numéricas e categóricas
num_attributes = Xtrain_uds.select_dtypes(include=['int64', 'float64'])
cat_attributes = Xtrain_uds.select_dtypes(exclude=['int64', 'float64'])

In [136]:
encoder = TargetEncoder(cols=cat_attributes.columns)
Xtrain_uds = encoder.fit_transform(df_target_train.drop('Attrition',axis=1),df_target_train.Attrition);
Xtest_uds  = encoder.transform(df_target_encoding_test.drop('Attrition',axis=1),df_target_encoding_test.Attrition);
ytrain_uds = df_target_train.Attrition
ytest_uds = df_target_encoding_test.Attrition

In [137]:
param_grid = {'C': np.arange(0.01,0.05,0.01)}
search_logistic = GridSearchCV(LogisticRegression(penalty='l1',solver='liblinear'), param_grid)
search_logistic.fit(Xtrain_uds,ytrain_uds)
predlogistic = search_logistic.predict(Xtest)

In [138]:
# Média de acurácia
search_logistic.score(Xtest, ytest)

In [139]:
# Matriz de confusão
matriz_confusao = confusion_matrix(ytest, predlogistic)
pd.DataFrame(matriz_confusao,index=['Real negativo','Real Verdadeiro'],columns=['Previsto Negativo','Previsto Positivo'])

In [140]:
# Outras métricas
print(classification_report(ytest, predlogistic))

In [141]:
# Computa probabilidades
y_pred_prob = search_logistic.predict_proba(Xtest)[:,1]

# Gera fpr, tpr e thresholds
fpr, tpr, thresholds = roc_curve(ytest, y_pred_prob)
auc_value = roc_auc_score(ytest, y_pred_prob)
plt.plot(fpr, tpr, color='blue', label='ROC curve (area = %0.2f)' % auc_value)
# curva ROC
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

## Random Oversampling

In [142]:
df_target_encoding = df_original.copy()

In [143]:
class_1_over = class_1.sample(class_count_0, replace=True)

test_over = pd.concat([class_1_over, class_0], axis=0)

sns.barplot(x=test_over['Attrition'].value_counts().index,y=test_over['Attrition'].value_counts().values).set_title('Contagem Target - Após Oversampling');

In [144]:
df = test_over.copy()
# Realizando split dos dados
df_target_encoding = df.copy()
Xtrain_over = df_target_encoding.drop('Attrition',axis=1)
ytrain_over = df_target_encoding.Attrition

In [145]:
param_grid = {'C': np.arange(0.01,0.05,0.01)}
search_logistic = GridSearchCV(LogisticRegression(penalty='l1',solver='liblinear'), param_grid)
search_logistic.fit(Xtrain_over,ytrain_over)
predlogistic = search_logistic.predict(Xtest)

In [146]:
# Média de acurácia
search_logistic.score(Xtest, ytest)

In [147]:
# Outras métricas
print(classification_report(ytest, predlogistic))

In [148]:
# Computa probabilidades
y_pred_prob = search_logistic.predict_proba(Xtest)[:,1]

# Gera fpr, tpr e thresholds
fpr, tpr, thresholds = roc_curve(ytest, y_pred_prob)
auc_value = roc_auc_score(ytest, y_pred_prob)
plt.plot(fpr, tpr, color='blue', label='ROC curve (area = %0.2f)' % auc_value)
# curva ROC
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

### Imbalanced-learn python module

In [149]:
import imblearn

#### Random under-sampling with imblearn

In [150]:
# import library
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

df_target_encoding = df_original.copy()

rus = RandomUnderSampler(random_state=42, replacement=True)# fit predictor and target variable
x_rus, y_rus = rus.fit_resample(Xtrain, ytrain)

print('original dataset shape:', Counter(ytrain))
print('Resample dataset shape', Counter(y_rus))

In [151]:
df_under = pd.concat([x_rus,y_rus],axis=1)
df = df_under.copy()
# Realizando split dos dados
df_target_encoding = df.copy()
Xtrain_uds = df_target_encoding.drop('Attrition',axis=1)
ytrain_uds = df_target_encoding.Attrition

In [152]:
param_grid = {'C': np.arange(0.01,0.05,0.01)}
search_logistic = GridSearchCV(LogisticRegression(penalty='l1',solver='liblinear'), param_grid)
search_logistic.fit(Xtrain_uds,ytrain_uds)
predlogistic = search_logistic.predict(Xtest)

In [153]:
# Média de acurácia
search_logistic.score(Xtest, ytest)

In [154]:
# Outras métricas
print(classification_report(ytest, predlogistic))

In [155]:
# Computa probabilidades
y_pred_prob = search_logistic.predict_proba(Xtest)[:,1]

# Gera fpr, tpr e thresholds
fpr, tpr, thresholds = roc_curve(ytest, y_pred_prob)
auc_value = roc_auc_score(ytest, y_pred_prob)
plt.plot(fpr, tpr, color='blue', label='ROC curve (area = %0.2f)' % auc_value)
# curva ROC
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

#### Random over-sampling with imblearn

In [156]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)

df_target_encoding = df_original.copy()
x_ros, y_ros = ros.fit_resample(Xtrain, ytrain)

print('Original dataset shape', Counter(ytrain))
print('Resample dataset shape', Counter(y_ros))

In [157]:
df_over = pd.concat([x_ros,y_ros],axis=1)
Xtrain_over = df_over.drop('Attrition',axis=1)
ytrain_over = df_over.Attrition

In [158]:
param_grid = {'C': np.arange(0.01,0.05,0.01)}
search_logistic = GridSearchCV(LogisticRegression(penalty='l1',solver='liblinear'), param_grid)
search_logistic.fit(Xtrain_over,ytrain_over)
predlogistic = search_logistic.predict(Xtest)

In [159]:
# Média de acurácia
search_logistic.score(Xtest, ytest)

In [160]:
# Outras métricas
print(classification_report(ytest, predlogistic))

In [161]:
# Computa probabilidades
y_pred_prob = search_logistic.predict_proba(Xtest)[:,1]

# Gera fpr, tpr e thresholds
fpr, tpr, thresholds = roc_curve(ytest, y_pred_prob)
auc_value = roc_auc_score(ytest, y_pred_prob)
plt.plot(fpr, tpr, color='blue', label='ROC curve (area = %0.2f)' % auc_value)
# curva ROC
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

#### Under-sampling: Tomek links

In [162]:
from imblearn.under_sampling import TomekLinks

df_target_encoding = df_original.copy()
tl = TomekLinks(sampling_strategy='majority')

# fit predictor and target variable
x_tl, y_tl = tl.fit_resample(Xtrain, ytrain)

print('Original dataset shape', Counter(ytrain))
print('Resample dataset shape', Counter(y_tl))

In [163]:
df_over = pd.concat([x_tl,y_tl],axis=1)
df = df_over.copy()
# Realizando split dos dados
df_target_encoding = df.copy()
Xtrain_tomek = x_tl.copy()
ytrain_tomek = y_tl.copy()

In [164]:
param_grid = {'C': np.arange(0.01,0.05,0.01)}
search_logistic = GridSearchCV(LogisticRegression(penalty='l1',solver='liblinear'), param_grid)
search_logistic.fit(Xtrain_tomek,ytrain_tomek)
predlogistic = search_logistic.predict(Xtest)

In [165]:
# Média de acurácia
search_logistic.score(Xtest, ytest)

In [166]:
# Outras métricas
print(classification_report(ytest, predlogistic))

In [167]:
# Computa probabilidades
y_pred_prob = search_logistic.predict_proba(Xtest)[:,1]

# Gera fpr, tpr e thresholds
fpr, tpr, thresholds = roc_curve(ytest, y_pred_prob)
auc_value = roc_auc_score(ytest, y_pred_prob)
plt.plot(fpr, tpr, color='blue', label='ROC curve (area = %0.2f)' % auc_value)
# curva ROC
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

#### SMOTE

In [168]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()

# fit predictor and target variable
x_smote, y_smote = smote.fit_resample(Xtrain, ytrain)

print('Original dataset shape', Counter(ytrain))
print('Resample dataset shape', Counter(y_smote))

In [169]:
df_smote = pd.concat([x_smote,y_smote],axis=1)
df = df_smote.copy()
# Realizando split dos dados
df_target_encoding = df.copy()
Xtrain_smote = x_smote.copy()
ytrain_smote = y_smote.copy()

In [170]:
param_grid = {'C': np.arange(0.01,0.05,0.01)}
search_logistic = GridSearchCV(LogisticRegression(penalty='l1',solver='liblinear'), param_grid)
search_logistic.fit(Xtrain_smote,ytrain_smote)
predlogistic = search_logistic.predict(Xtest)

In [171]:
# Média de acurácia
search_logistic.score(Xtest, ytest)

In [172]:
# Outras métricas
print(classification_report(ytest, predlogistic))

In [173]:
# Computa probabilidades
y_pred_prob = search_logistic.predict_proba(Xtest)[:,1]

# Gera fpr, tpr e thresholds
fpr, tpr, thresholds = roc_curve(ytest, y_pred_prob)
auc_value = roc_auc_score(ytest, y_pred_prob)
plt.plot(fpr, tpr, color='blue', label='ROC curve (area = %0.2f)' % auc_value)
# curva ROC
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

### NearMiss

In [174]:
from imblearn.under_sampling import NearMiss

nm = NearMiss()

x_nm, y_nm = nm.fit_resample(Xtrain, ytrain)

print('Original dataset shape:', Counter(ytrain))
print('Resample dataset shape:', Counter(y_nm))

In [175]:
df_nm = pd.concat([x_nm,y_nm],axis=1)
df = df_nm.copy()
# Realizando split dos dados
df_target_encoding = df.copy()
Xtrain_nm = x_nm.copy()
ytrain_nm = y_nm.copy()

In [176]:
param_grid = {'C': np.arange(0.01,0.05,0.01)}
search_logistic = GridSearchCV(LogisticRegression(penalty='l1',solver='liblinear'), param_grid)
search_logistic.fit(Xtrain_nm,ytrain_nm)
predlogistic = search_logistic.predict(Xtest)

In [177]:
# Média de acurácia
search_logistic.score(Xtest, ytest)

In [178]:
# Outras métricas
print(classification_report(ytest, predlogistic))

In [179]:
# Computa probabilidades
y_pred_prob = search_logistic.predict_proba(Xtest)[:,1]

# Gera fpr, tpr e thresholds
fpr, tpr, thresholds = roc_curve(ytest, y_pred_prob)
auc_value = roc_auc_score(ytest, y_pred_prob)
plt.plot(fpr, tpr, color='blue', label='ROC curve (area = %0.2f)' % auc_value)
# curva ROC
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

### Penalize Algorithms (Cost-Sensitive Training)

In [180]:
from sklearn.svm import SVC

# we can add class_weight='balanced' to add panalize mistake
svc_model = SVC(class_weight='balanced', probability=True)

svc_model.fit(Xtrain, ytrain)

svc_predict = svc_model.predict(Xtest)# check performance
print('ROCAUC score:',roc_auc_score(ytest, svc_predict))
print('Accuracy score:',accuracy_score(ytest, svc_predict))
print('F1 score:',f1_score(ytest, svc_predict))

### Random Forest Classifier

In [183]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()

# fit the predictor and target
rfc.fit(Xtrain, ytrain)

# predict
rfc_predict = rfc.predict(Xtest)# check performance
print('ROCAUC score:',roc_auc_score(ytest, rfc_predict))
print('Accuracy score:',accuracy_score(ytest, rfc_predict))
print('F1 score:',f1_score(ytest, rfc_predict))

### Considerações - Modelo de Machine Learning
Diante dessas opções de tratamento para dados desbalanceados, nota-se que a técnica: Random over-sampling with imblearn foi a que produziu melhores resultados. Para análise do desempenho dos modelos foram utilizadas as métricas: acurácia, f1 score e área da curva ROC.

# Resumo

Neste projeto foi realizada uma análise exploratória de dados na qual foram obtidas métricas descritivas, análise da distribuição dos dados, foram respondidas algumas hipóteses. Além disso, foram observadas o grau de correlação entre as variáveis e, por fim, desenvolveu-se um modelo de machine learning para prever o desligamento do funcionário baseado nas demais features. Em virtude do desbalanceamento dos dados na variável target foram utilizadas diversas técnicas para tratar este desbalanceamento, em seguida realizou-se uma comparação utilizando algumas métricas e, finalmente, foi escolhido um modelo para este problema.